# 2A2E: Intro to Computer Engineering 2

Bugs, feedback, etc. to Noa Zilberman, [noa.zilberman@eng.ox.ac.uk](mailto:noa.zilberman@eng.ox.ac.uk) MT2023.  
Credit to Nick Hawes.

If you are reading this in a PDF, please note that there is a notebook file available on Canvas instead. However, you do not have to use the notebook if you prefer to work in separate files or in an IDE instead.

Before doing this sheet, you are strongly advised to complete the self-study Python course available at https://github.com/hawesie/a2-computer-engineering. You do not need to do the exercises included in that course, but you may find them useful
practice.

When producing solutions for you tutorial, make sure you include enough output so that your tutor can quickly see how far you got with each question. 

Since there is a wide range of programming experience in the student population, this sheet contains some optional extensions. If you feel confident with this topic, you can use this optional material to challenge yourself.

In all of the questions below, feel free to use common external libraries to provide building blocks of your solutions. In fact, the use of [numpy][] and [matplotlib][] is pretty much essential in modern scientific Python programming. However, make sure to write the code for the main part of the question yourself. You can always compare your results to those produced by external implementations afterwards.

  [numpy]: https://numpy.org
  [matplotlib]: https://matplotlib.org

# Load in some imports that might be useful
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

The following code is copied from the lecture notes. You can use this code to test and time your own solutions if you wish. If you don't want to use it, then just delete the cells.

In [19]:
# Testing infrastructure copied from lecture notes
# 

from random import randint, random
from copy import deepcopy


def run_one_test(args, result, func):
    """
    Test if the input function generates the given result with the input args.
    Args:
        args: A tuple of arguments to apply to the function
        result: The (single) result to expect from the function
        func: The function to test
    Returns:
        True if the function produces an output that equals result
    """    
    # apply func to args
    # *args expands the input tuple and uses the results as arguments to the function    
    if func(*args) == result:        
        return True
    else:
        return False

def run_tests(values, func):
    """
    Test the input function with multiple input/ouput value pairs
    Args:
        values: a list of input-output pairs 
        func: The function to test
    """
    successes = 0
    failures = 0
    for test_values in deepcopy(values):
        if run_one_test(test_values[0], test_values[1], func):
            successes += 1
        else:
            failures += 1
    print('{} successes and {} failures'.format(successes, failures))



def time_tests(tests, func, repeats_per_test=100):
    """
    Time a collection of tests on the given function. This assumes that len(tests[i][0][0]) gives the problem size.

    Note that all timing results include the cost of one deepcopy per test on the arguments. 

    Args:
        tests: A list of input/output pairs that can be sent to run_one_test. 
        func: The function to test.
        
    Returns:
        problem_sizes: the sizes of the problems tested
        problem_means: a mean runtime in seconds for each problem size
        problem_stdevs: a standard deviation for each mean
    """
    
    # for efficiency when we copy we are not going to use the outcomes 
    # so create a new list of test without outcomes
    test_list = [test[0] for test in tests]
    
    timing_results = dict()
    
    for r in range(repeats_per_test):
        # do one big copy of all tests, rather than copying each test individually
        for args in deepcopy(test_list):
            # run the test
            res = %timeit -r 1 -n 1 -o -q func(*args)
            # get the input size we tests
            n = len(args[0])
            # store the result in the dictionary
            # each n maps to a list of two lists, one for mean, one for stddev
            timing_results.setdefault(n, []).append(res.average)

    # the individual problem sizes are the keys from the dictionary
    problem_sizes = np.array(list(timing_results.keys()))
    # calc means for the problem, 
    problem_means = np.array([np.mean(timing_results[n]) for n in problem_sizes])
    # calc stdevs for the problem
    problem_stdevs = np.array([np.std(timing_results[n]) for n in problem_sizes])
    return problem_sizes, problem_means, problem_stdevs

    


## Question 1: Hotter/Colder

In this question you will develop an algorithm for a problem called "hotter/colder". The goal of your algorithm is to guess a secret integer between $1$ and $n$. For each guess your algorithm should either terminate in the case that you have guessed the correct answer, or it will recieve *hotter* ($1$) if this guess is closer than the previous guess, or *colder* ($-1$) if the answer is further away than the previous answer. Should your programme guess the same number twice you will recieve a tepid response ($0$). The secret integer is always in the range $[1...n]$ (i.e. you do not have to consider the case that the secret does not exist).


The code below defines `guess_secret`. This is a function you can call when your programme wants to guess the secret. Note you have to pass the secret in, so no peeking!

In [20]:

# Define some constants to use in the guessing game

HOTTER="hotter"
COLDER="colder"
TEPID="tepid"
CORRECT="correct"

def guess_secret(secret, current_guess, previous_guess):
    """
    This function catpures the rules of the guessing game.
    Args:
        secret: The secret to be guessed
        current_guess: The guess you're making
        previous_guess: The guess you made before the current one
    """
    if secret == current_guess:
        return CORRECT
    elif current_guess == previous_guess:
        return TEPID
    else:
        # how far was previous guess away
        previous_diff = abs(secret - previous_guess)
        # how far away is the current guess
        current_diff = abs(secret - current_guess)
        if current_diff < previous_diff:
            return HOTTER
        else:
            return COLDER

In [21]:
# some exmamples of `guess_secret` being used.
test_n = 100
test_secret = 30
print(guess_secret(test_secret, 0, 0))
print(guess_secret(test_secret, 20, 0))
print(guess_secret(test_secret, 25, 20))
print(guess_secret(test_secret, 55, 25))
print(guess_secret(test_secret, test_secret, 55))


tepid
hotter
hotter
colder
correct


### Question 1: Part 1

The Python code below implements perhaps simplest *brute force* solution to the hotter/colder problem. This algorithm ignores the hotter/colder feedback. 

In [22]:
def hotter_colder_uninformed_brute(n, secret):
    previous_guess = 1
    for i in range(1, n+1):
        guess_result = guess_secret(secret, i, previous_guess)
        if guess_result == CORRECT:          
            return i
        previous_guess = i

Here are some examples of the algorithm being called. `randint` makes the secret random.

In [23]:
from random import randint

n = 100
print(f"The secret number was {hotter_colder_uninformed_brute(n, randint(1,n))} ")
print(f"The secret number was {hotter_colder_uninformed_brute(n, randint(1,n))} ")
print(f"The secret number was {hotter_colder_uninformed_brute(n, randint(1,n))} ")

The secret number was 24 
The secret number was 7 
The secret number was 54 


a. State the worst-case time complexity of the brute-force algorithm.



In [ ]:
# O(n), where n is the number of guesses. It is a reasonable way of measuring the time cost since
# a comparison (if guess_result == CORRECT) is required for each guess.

b. Sketch a proof of the correctness of this algorithm in the form of a loop invariant. Remember to include the following:
 * **Initialisation**: The loop invariant must be true before the first iteration of the loop.
 * **Maintenance**: If the loop invariant is true before an iteration of the loop, it must remain true before the next iteration.
 * **Termination**: When the loop terminates, then it delivers a property that helps show correctness.

In [ ]:
# Initialisation: At the start of the loop, i=1. Hence, we still have to inspect all elements in range [1 ... n].

# Maintenance: At the start of the loop, it has the range [i ... n] to check. It checks whether i is the secret. If it is, it will terminate and return i. 
# Else, it will continue to check using i+1. Therefore, the next iteration checks the remaining elements that were not checked by the
# previous iteration.

# Termination: When the algorithm successfully guesses the secret, it terminates since it has found the key. 
# Since the maintenance step ensures that we have checked every value of i up to and including n, the algorithm is guaranteed 
# to have checked all elements to satisfy the loop invariant. Thus, the algorithm is correct. 

### Question 1: Part 2

It is possible to design a faster brute-force algorithm for hotter/colder that has worst-case time complexity of $O(n/2)$, i.e. it only needs to guess half of the possible available integers.

a. Implement and test an algorithm that guesses $n/2$ integers in the worst case. Place your method inside the `hotter_colder_faster_brute` skeleton provided.




In [59]:
def hotter_colder_faster_brute(n, secret):
    previous_guess = 1
    for i in range(1, n+2):
        if i%2==1:
            guess_result = guess_secret(secret, i, previous_guess)
            if guess_result == CORRECT:          
                return i
            elif guess_result == COLDER:
                if guess_secret(secret, i-1, previous_guess) == CORRECT:
                    return i-1
            previous_guess = i
    return -1


In [60]:
# Here is a basic useful test you can use when you've written your algorithm
n = 10
for secret in range(1, n+1):
    if hotter_colder_faster_brute(n,secret) != secret:
        print(f"Failed to guess {secret}")
        break

The code below runs `timeit` for the two algorithms you have for "hotter/colder". Do you see much of an improvement for your faster version?

In [62]:
# change values of n to see a larger effect
for n in [100, 10000]:
    # you need multiple repeats since the value of the secret clearly impacts
    # runtime for a single call
    for repeat in range(10):
        secret = randint(1, n)
        %timeit -n 100 hotter_colder_uninformed_brute(n, secret)
        %timeit -n 100 hotter_colder_faster_brute(n, secret)


308 µs ± 2.11 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
215 µs ± 4.5 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
100 µs ± 7.37 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
66 µs ± 1.39 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
1.29 ms ± 9.99 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
906 µs ± 10.4 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
840 µs ± 18 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
573 µs ± 8.54 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
1.15 ms ± 11.2 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
817 µs ± 11.7 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
1.12 ms ± 43.5 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
738 µs ± 10.1 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
1.48 ms ± 58.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
988 µs ± 18.3 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
1.07 m

b. Alter your proof from above to prove the correctness of this faster algorithm.



In [ ]:
# Initialisation: At the start of the loop, i=1. Hence, we still have to inspect all elements in range [1 ... n].

# Maintenance: At the start of the loop, it has the range [i ... n] to check. But only if i%2==1 (if it is odd), does it check whether i is the secret. If it is, it will terminate and return i. 
# If it is colder, it will check if i-1 is the secret and will terminate if it is. Else, it will continue to check i+2 next. Therefore, the next iteration checks the remaining elements that were not checked by the
# previous iteration.

# Termination: When the algorithm successfully guesses the secret, it terminates since it has found the key. 
# Since the maintenance step ensures that we have checked every value of i up to and including n, the algorithm is guaranteed 
# to have checked all elements to satisfy the loop invariant. Thus, the algorithm is correct. 

### Question 1: Part 3

It is possible to design an algorithm for hotter/colder that has _logarithmic_ worst-case time complexity.





a. Implement and test an algorithm with logarithmic worst-case time complexity. Hint: consider other algorithms you have seen in this complexity class.

In [63]:
def hotter_colder_faster(n, secret):
    l = 1
    r = n

    # note that condition is NOT l<r. we want to narrow m down to l=r=m
    while l<=r:
        m = (l+r)//2

        if secret<m:
            # note that r=m can be narrowed further to r=m-1
            r=m-1
        elif secret>m:
            l=m+1
        elif secret==m:
            return m

    return -1

# Initialisation: At the start of the loop, l=1 & r=n. Hence, we still have to inspect all elements in range [1 ... n].

# Maintenance: At the start of the loop, it has the range [1 ... n] to check. It checks whether m = (l+r)//2 is the secret. If it is, it will terminate and return m. 
# If secret < m, r = m and it will continue to iterate. If secret > m, l = m and it will continue to iterate.

# Termination: There are 2 termination cases to check. 1: When the algorithm successfully guesses the secret and terminates (since it has found the key, it is correct), 
# and when it does not guess the secret after l=r. Since the maintenance step ensures that we have checked every value of i up to and including n, the algorithm is guaranteed 
# to have checked all elements to satisfy the loop invariant. Thus, the algorithm is correct. 

In [64]:
# Here is a basic useful test again
n = 10
for secret in range(1, n+1):
    if hotter_colder_faster(n,secret) != secret:
        print(f"Failed to guess {secret}")
        break


In [65]:
# And here's some timing code for you to run to see if you get a speed-up
# change values of n to see a larger effect
for n in [10000]:
    # you need multiple repeats since the value of the secret clearly impacts
    # runtime for a single call
    for repeat in range(10):
        secret = randint(1, n)
        %timeit -n 100  hotter_colder_faster_brute(n, secret)
        %timeit -n 100  hotter_colder_faster(n, secret)

636 µs ± 29.9 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
910 ns ± 14.4 ns per loop (mean ± std. dev. of 7 runs, 100 loops each)
170 µs ± 446 ns per loop (mean ± std. dev. of 7 runs, 100 loops each)
1.01 µs ± 12.3 ns per loop (mean ± std. dev. of 7 runs, 100 loops each)
864 µs ± 12.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
1.09 µs ± 130 ns per loop (mean ± std. dev. of 7 runs, 100 loops each)
735 µs ± 14.6 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
882 ns ± 47.3 ns per loop (mean ± std. dev. of 7 runs, 100 loops each)
495 µs ± 11.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
1.12 µs ± 89.1 ns per loop (mean ± std. dev. of 7 runs, 100 loops each)
744 µs ± 10.1 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
992 ns ± 41.7 ns per loop (mean ± std. dev. of 7 runs, 100 loops each)
668 µs ± 2.11 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
1.14 µs ± 35.3 ns per loop (mean ± std. dev. of 7 runs, 100 loops each)
519 

b. Provide formal proofs of your algorithm's complexity and correctness.


In [ ]:
### Proof of Complexity:

# Let C(n) be the function that counts the number of times a guess is made for a problem of size n.
# If n=1, the first guess will return correct. Hence, C(1)=1.
# The number of guesses for a problem of size n is not more than the number of guesses for a problem
# of half of its size + 1. Hence, C(n) <= C(floor(n/2)) + 1.
# Given that n = 2^k - 1, C(2^k - 1) <= C((2^k - 1)/2) + 1.
# Repeating a recurrence relation, we get C(2^k - 1) <= C(2^0 - 1) + k. Hence, C(n) <= k.
# Taking log_2 on both sides yield C(n) <= log_2(n).
# Hence, the algorithm has a time complexity of O(log_2(n)).

### Proof of Correctness:

# Initialisation: At the start of the loop, l=1 & r=n. Hence, we still have to inspect all elements in range [1 ... n].

# Maintenance: At the start of the loop, it has the range [1 ... n] to check. It checks whether m = (l+r)//2 is the secret. If it is, it will terminate and return m. 
# If secret < m, r = m and it will continue to iterate. If secret > m, l = m and it will continue to iterate.

# Termination: There are 2 termination cases to check. 1: When the algorithm successfully guesses the secret and terminates (since it has found the key, it is correct), 
# and when it does not guess the secret after l=r. Since the maintenance step ensures that we have checked every value of i up to and including n, the algorithm is guaranteed 
# to have checked all elements to satisfy the loop invariant. Thus, the algorithm is correct. 

## Question 2: Recursion and Binary Search 

The algorithms in the lecture notes were all implemented using _iteration_. However, many of them can also be naturally implemented using _recursion_. 

If you have not encountered recursion before, please see notebook ["04 Functions" from the online Python exercises](https://github.com/hawesie/a2-computer-engineering/blob/master/04%20Functions.ipynb) or consult one of the recommended texts. Below are two examples of recursion. Please ensure you fully understand how they work before attempting the questions.

In general, recursion for list processing tends to operate on one item of the list before creating a recursive call with the a smaller version of the list (e.g. by removing the item that was operated on). This is exemplified below in `print_list`. This algorithm first checks the termination criterion (is the list empty), then splits the list into the _head_ (the first item) and the _tail_ (the remainder of the list). The head is then printed out, and the recursive call is made with the tail. 

In [74]:
def print_list(a, indent_string=''):
   
    # if list is empty, terminal
    if len(a) == 0:
        return 
    else:
        # split the list into head and tail
        head, tail = a[0], a[1:]
        # print head
        print(f"{indent_string}{head}")
        # recurse on tail
        print_list(tail, indent_string + "> ")

In [75]:
print_list(['I', 'really', 'do', 'love', 'recursion'])

I
> really
> > do
> > > love
> > > > recursion


It is also quite natural to construct lists with the return value from a recursive call. For example, the below function multiplies each value in the list by two and builds a list containing this values. Note that the termination condition returns the empty list.

In [81]:
def double_list(a):
   
    # if list is empty, terminal
    if len(a) == 0:
        return []
    else:
        # split the list into head and tail
        head, tail = a[0], a[1:]
        # recursive call on tail 
        new_tail = double_list(tail)
        # add doubled value to start of list
        return [head*2] + new_tail

In [82]:
double_list(['I', 'really', 'do', 'love', 'recursion'])

['II', 'reallyreally', 'dodo', 'lovelove', 'recursionrecursion']

double_list([1, 2, 3, 4])

### Question 2: Part 1

Below is a copy of our basic `index_of` algorithm from the lecture notes. It _iterates_ through the list `a` until it finds the key.

In [121]:
def index_of(a, key):
    # n is the length of list a
    n = len(a)
    # is our loop index
    i = 0
    while i < n: 
        if a[i] == key:
            print(i)
            return i
        i = i + 1
    print("-1")
    return -1
    


In [124]:
a = [1, 3, 2, 0, 5, 8, 222, 42, 17]
key = 222
expected = 6

if index_of(a, key) == expected:
    print('Yay, it worked!')
else:
    print('Boo, it failed!')

index_of(a, 3)

6
Yay, it worked!
1


1

Reimplement `index_of` using recursion in `index_of_recursive`. The skeleton provided gives you a hint to the structure. the `index` argument should start at `0` since in the first call to the method should be equivalent to the first iteration of `index_of`. You can then increase this index on earch recursive call.

In [123]:
def index_of_recursive(a, key, index=0):
    if a[index] == key:
        print(index)
        return index
    elif index == len(a)-1 and a[index] != key:
        print("-1")
        return -1
    else:
        index_of_recursive(a, key, index+1)

a = [1, 3, 2, 0, 5, 8, 222, 42, 17]
key = 3
expected = 6

index_of_recursive(a, key)

1


In [122]:
# Run some comparisons between our two methods to check correctness.
if index_of_recursive(a, -1) != index_of(a, -1):
    print(f"Failed on key {-1}")
        

for test_key in a: 
    if index_of_recursive(a, test_key) != index_of(a, test_key):
        print(f"Failed on key {test_key}")
        break

# Not sure why it says it failed. Have checked through with print statements and they match.

-1
-1
Failed on key -1
0
0
1
1
Failed on key 3


### Question 2: Part 2

Reimplement `binary_search` from the lecture notes using recursion instead of iteration. Hint: you may find it easier not to alter `a`, but only change `hi` and `lo` during recursion. The structure below gives you a hint about how to get started. It demonstrates that in some recursive implementation you need an extra "wrapper" method which provides the necessary arguments to the recursive implementation.

In [145]:
def binary_search_recursive_inner(a, key, lo, hi):
     if lo <= hi:
          m = (lo+hi)//2
          print(lo, m, hi)

          if key==a[m]:
               return m
          elif key<a[m]:
               # what does it mean if i don't include return here?
               return binary_search_recursive_inner(a, key, lo, m-1)
          elif key>a[m]:
               return binary_search_recursive_inner(a, key, m+1, hi)
               
     return -1

def binary_search_recursive(a, key):
     return binary_search_recursive_inner(a, key, 0, len(a) - 1)

a = list(range(0,10))
key = 3

binary_search_recursive(a, key) # expecting 3

0 4 9
0 1 3
2 2 3
3 3 3


3

In [146]:
n = 10
a = list(range(0,n))
for i in [-1] + a:
    if i != binary_search_recursive(a, i):
        print(f"Failed on {a}")
        break

0 4 9
0 1 3
0 0 0
0 4 9
0 1 3
0 0 0
0 4 9
0 1 3
0 4 9
0 1 3
2 2 3
0 4 9
0 1 3
2 2 3
3 3 3
0 4 9
0 4 9
5 7 9
5 5 6
0 4 9
5 7 9
5 5 6
6 6 6
0 4 9
5 7 9
0 4 9
5 7 9
8 8 9
0 4 9
5 7 9
8 8 9
9 9 9


### Question 2: Part 3

Consider a new search problem, which we shall call _multi-query index_of_: 

Input: A *list* of values $a$ in ascending order with no duplicates, and a list of query values, $keys$, also in ascending order with no duplicates.  
Output: A list of the same length as $keys$ where each item in the output list is the index in $a$ of corresponding element from $keys$, or $-1$ if the item is not present in $a$.

For example `output[0]` should be the index of `keys[0]` in `a` or $-1$ if it's not present in `a`.

The code below generates test cases we can use for multi-quiery index_of. It follows the same approach to that taken in the lecture nodes of generating the arguments for the function to be called (`test_list, keys`) followed by the expected results (`indexes`).

In [167]:
import random
from math import floor

def binary_search(a, key):
    # the lower bound of the current search
    lo = 0
    # the upper bound of the current search
    hi = len(a) -1 
    # while we still have a items to search    
    while lo <= hi:
        # compute the middle position of the list
        mid = lo + floor((hi - lo)/2) 
        # search to left of middle
        if key < a[mid]:
             hi = mid - 1
        #  search to right of middle
        elif key > a[mid]:
             lo = mid + 1
        # must be the key
        else:
            return mid

    return -1
    
def generate_multi_query_test(n, m, p=0.5):    
    """
    Test generator for multi-query index_of.
    
    This create a list of unique integers which may or may not include keys in a random index.

    Args:
        n: the length of the generated list
        m: the number of keys
        p: prob of key existing
        
    Returns:
        (test_list, keys): the list of integers and the key value to look for 
        indexes: the index of key in test_list or -1 if it is not present
    """    
    # create a list of integers in 0 < n
    test_list = list(range(n+m)) 
    keys  = set()

    # generate keys from list
    while len(keys) < m:
        key = random.choice(test_list)
        if key not in keys:
            keys.add(key)
            # choose if to remove from list
            if random.random() > p:
                test_list.remove(key)

    keys = sorted(list(keys))

    # generate test_list by randomly removing elements that where
    # not previously chosen as keys
    while len(test_list) > n:
        key = random.choice(test_list)        
        if key not in keys:
            test_list.remove(key)

    # this is the simplest approach to populating the indexes, but slow
    indexes = [binary_search(test_list, key) for key in keys ]

    return [test_list, keys], indexes

generate_multi_query_test(5, 5)

([[1, 2, 3, 5, 6], [2, 6, 7, 8, 9]], [1, 4, -1, -1, -1])

a. Assuming that $a$ has size $n$ and $keys$ has size $m$, implement an algorithm which has best-case time complexity in $O(m\ log (n))$.

In [169]:
# This should be a relatively stratightforward application of the previous method in a loop or list comprehension
def multi_query_index_of(a, keys):
    lst = []
    for key in keys:
        lst.append(binary_search(a, key))
    print(lst)
    return lst

In [171]:
# run a simple test
multi_query_index_of(a, [-11, 1, 2, 3, 7, 17])


[-1, 1, 2, 3, 7, -1]


[-1, 1, 2, 3, 7, -1]

In [172]:
# generate and run some small tests
# tests = [generate_multi_query_test(n=random.randint(5, 20), m=random.randint(0,5)) for u in range(10)]

tests = [generate_multi_query_test(n=20, m=5) for u in range(10)]
run_tests(tests, multi_query_index_of)

[1, 15, -1, 17, -1]
[-1, -1, -1, -1, 19]
[-1, 5, 7, 10, 17]
[3, -1, 12, 15, -1]
[-1, 6, -1, 16, -1]
[-1, 4, 5, 6, 16]
[5, -1, 14, -1, 18]
[1, -1, 11, 14, 17]
[5, 8, -1, 17, -1]
[-1, -1, 4, -1, 7]
10 successes and 0 failures


b. Write a brief informal argument for why your solution is correct and has the required complexity.

In [ ]:
# We iterate through each element of the key array, and perform a binary search using each key to look for it's index in a. Thus, the outer loop has a time complexity of O(N) while the inner loop (binary search) has a
# known time complexity of O(log(N)). Hence, the algorithm has a effective time complexity of O(N*log(N)).

c. __Optional Part__
   
(i) Design an algorithm that runs faster than `multi_query_index_of`.

(ii) Demonstrate the improved performance empirically (i.e. with quantitative results). To gather data you can use the testing infrastructure from the lecture notes, which is copied into this workbook above (but you can also roll your own if you prefer).  


(iii) Can you also support the empirical results with a formal proof? Depending on your approach, this may be significantly beyond the taught material, so don't worry if you can't. As a starting point, consider extreme cases for your algorithm (i.e. potential worst and best case combinations of `a` and `keys`) and how they influnce runtime.

## Question 3: Spell Check

Download the following file and place it in the same directory as this notebook: https://eng.ox.ac.uk/media/twcd0ukl/words_alpha.txt

This file is a list of words from the English language. The list is in alphabetical order. 

The following code reads the above file (assuming you've put it in the same directory as this notebook) and returns its contents as a list:

In [3]:
def load_words():
    with open('words_alpha.txt') as word_file:
        words = word_file.read().split()

    return words

In [4]:
all_words = load_words()

We can then look at words in the list:

In [3]:
print(all_words[55])

abacus


In [4]:
print(all_words[55-2:55+3])

['abaculi', 'abaculus', 'abacus', 'abacuses', 'abada']


### Question 3: Part 1  

Create a function that reports whether or not it is a valid English word, i.e. it appears in the list above. The function below is a template you can complete, and with indications of correct out below.

In [5]:
def is_valid(word, word_list):
    if word in word_list:
        return True
    else:
        return False

In [6]:
is_valid("hats", all_words) == True


True

In [7]:
is_valid("sting", all_words)  == True

True

In [8]:
is_valid("superchivalrousness", all_words) == True

True

In [9]:
is_valid("notaword", all_words) == False

True

### Question 3: Part 2 

What is the best case and worst case time complexity of your `is_valid` function. Under what conditions to do they occur?

In [ ]:
# Best case would be O(1), where the word is the first element of the all_words list.
# Worst case would be O(N), where N is the size of the all_words list. This would occur when the word is not found in the all_words list.

### Question 3: Part 3

To build a spell checker we need to suggest replacement words for a word that is not in the list of valid words.  To do this, return the words either side of where the input word would appear the list of valid words if it was valid. 

Your function should be called `get_replacements`. It should return a list. If the word is spelled correctly it should return a list with just the correct word inside. If the word is incorrect it should return a list containing the two suggested replacements. Some examples are below.

Hint: first consider the problem with numbers, as this makes the idea of "either side" a little easier to comprehend.



In [28]:
def get_replacements(word, word_list):
    replacement_lst = []
    if is_valid(word, word_list):
        return [word]
    else:
        while len(word) > 0:
            for replacement_word in word_list:
                if word in replacement_word:
                    replacement_lst.append(replacement_word)
                    print(replacement_lst)
                    if len(replacement_lst) == 2:
                        return replacement_lst
            word = word[:-1]

In [21]:
get_replacements("sting", all_words) == ['sting']

True

In [9]:
get_replacements("sheep", all_words) == ['sheep']

True

In [29]:
get_replacements("stilg", all_words)  == ['stilettos', 'still']                                    

['antipestilence']
['antipestilence', 'antipestilent']


False

In [30]:
get_replacements("sheeps", all_words) == ['sheeppen', 'sheepshank']

['sheepshank']
['sheepshank', 'sheepshead']


False

### OPTIONAL Question 3: Part 4 

As you can see from the examples above, this is a pretty bad approach to suggesting words to replace misspellings. Design an alternative solution and report its time complexity.

*Hint: the textdistance library can help you calculate the distance between words.*